<a href="https://colab.research.google.com/github/lucyzandile/unsupervised-predict-streamlit-template/blob/master/Zandile_Lushaba_Content_based_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction


## Problem Statement

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Data Preprocessing</a>

<a href=#four>4. Exploratory Data Analysis (EDA)</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="one"></a>
## Importing packages

In [ ]:
# Libraries for data loading, data manipulation and data visulisation

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from matplotlib import rc

import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<a id="two"></a>
## Loading the Data

In [ ]:
# importing datasets

train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")

test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")

#df_gscores = pd.read_csv("genome_scores.csv")

df_gtags = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/genome_tags.csv")

imdb = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/imdb_data.csv")

df_links = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/links.csv")

movies = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/movies.csv")

#df_tags = pd.read_csv("")

### Overview of the datasets

In [ ]:
#Checking df_train dataset head
display(train.head())

#Checking df_train dataset information
train.info()

#Checking for null values
display(train.isnull().sum())

In [ ]:
#Checking df_test dataset head
display(test.head())

#Checking df_test dataset information
test.info()

#Checking for null values
display(test.isnull().sum())

In [ ]:
#Checking df_gtags dataset head
display(df_gtags.head())

#Checking df_gtags dataset information
df_gtags.info()

In [ ]:
#Checking df_imdb dataset head
display(df_imdb.head())

#Checking df_imdb dataset information
df_imdb.info()

In [ ]:
#Checking df_links dataset head
display(df_links.head())

#Checking df_links dataset information
df_links.info()

In [ ]:
#Checking df_movies dataset head
display(df_movies.head())

#Checking df_movies dataset information
df_movies.info()

<a id="three"></a>
## Data Preprocessing

In [ ]:
# Create a copy of the train data

train_copy = train.copy()

df_train = train_copy

In [ ]:
# Create a copy of the test data

test_copy = test.copy()

df_test = test_copy

In [ ]:
# Create a copy of the test data

imdb_copy = imdb.copy()

df_imdb = imdb_copy

In [ ]:
# Create a copy of the test data

movies_copy = movies.copy()

df_movies = movies_copy

### Merging datasets

In [ ]:
# Merge the df_train and df_movies
df_merge = df_train.merge(df_movies, on='movieId')

#Checking df_merge dataset head
display(df_merge.head())

,userId,movieId,rating,timestamp,title,genres
0,5163,57669,4.0,1518349992,In Bruges (2008),Comedy|Crime|Drama|Thriller
1,87388,57669,3.5,1237455297,In Bruges (2008),Comedy|Crime|Drama|Thriller
2,137050,57669,4.0,1425631854,In Bruges (2008),Comedy|Crime|Drama|Thriller
3,120490,57669,4.5,1408228517,In Bruges (2008),Comedy|Crime|Drama|Thriller
4,50616,57669,4.5,1446941640,In Bruges (2008),Comedy|Crime|Drama|Thriller


In [ ]:
# Merge the df_merge and df_imdb
df_merge1 = df_movies.merge(df_imdb, on='movieId')

#Checking df_merge dataset head
display(df_merge1.head())

#Checking for null values
display(df_merge1.isnull().sum())

,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Jumanji (1995),Adventure|Children|Fantasy,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Grumpier Old Men (1995),Comedy|Romance,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Father of the Bride Part II (1995),Comedy,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


movieId              0
title                0
genres               0
title_cast        9665
director          9519
runtime          11345
budget           17583
plot_keywords    10482
dtype: int64

In [ ]:
df_merge1 = df_merge1[['title','genres', 'title_cast', 'director', 'plot_keywords']]
df_merge1.head()

,title,genres,title_cast,director,plot_keywords
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,toy|rivalry|cowboy|cgi animation
1,Jumanji (1995),Adventure|Children|Fantasy,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,board game|adventurer|fight|game
2,Grumpier Old Men (1995),Comedy|Romance,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,boat|lake|neighbor|rivalry
3,Waiting to Exhale (1995),Comedy|Drama|Romance,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,black american|husband wife relationship|betra...
4,Father of the Bride Part II (1995),Comedy,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,fatherhood|doberman|dog|mansion


In [ ]:
# convert data types to strings for string handling
df_merge1['genres'] = df_merge1.genres.astype(str)
df_merge1['title_cast'] = df_merge1.title_cast.astype(str)
df_merge1['director'] = df_merge1.director.astype(str)
df_merge1['plot_keywords'] = df_merge1.plot_keywords.astype(str)

# discarding the line between genres and putting the genres in a list of words
df_merge1['genres'] = df_merge1['genres'].map(lambda x: x.lower().split('|'))

# discarding the liness between the actors full names and getting only the first three names
df_merge1['title_cast'] = df_merge1['title_cast'].map(lambda x: x.split('|')[:3])

#converting name and surname of directors to lowercase
df_merge1['director'] = df_merge1['director'].apply(lambda x: "".join(x.lower() for x in x.split()))

# discarding the line between plot_keywords and putting the plot_keywords in a list of words
df_merge1['plot_keywords'] = df_merge1['plot_keywords'].map(lambda x: x.lower().split('|'))


# merging together first and last name for each title_cast and director, so it's considered as one word 
for index, row in df_merge1.iterrows():
    row['title_cast'] = [x.lower().replace(' ','') for x in row['title_cast']]
    row['director'] = ''.join(row['director']).lower()


In [ ]:
df_merge1.set_index('title', inplace = True)
df_merge1.head()

,genres,title_cast,director,plot_keywords
title,,,,
Toy Story (1995),"[adventure, animation, children, comedy, fantasy]","[tomhanks, timallen, donrickles]",johnlasseter,"[toy, rivalry, cowboy, cgi animation]"
Jumanji (1995),"[adventure, children, fantasy]","[robinwilliams, jonathanhyde, kirstendunst]",jonathanhensleigh,"[board game, adventurer, fight, game]"
Grumpier Old Men (1995),"[comedy, romance]","[waltermatthau, jacklemmon, sophialoren]",markstevenjohnson,"[boat, lake, neighbor, rivalry]"
Waiting to Exhale (1995),"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine]",terrymcmillan,"[black american, husband wife relationship, be..."
Father of the Bride Part II (1995),[comedy],"[stevemartin, dianekeaton, martinshort]",alberthackett,"[fatherhood, doberman, dog, mansion]"


In [ ]:
df_merge1['bag_of_words'] = ''
columns = df_merge1.columns
for index, row in df_merge1.iterrows():
    words = ''
    for col in columns:
        if col != 'director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df_merge1.drop(columns = [col for col in df_merge1.columns if col!= 'bag_of_words'], inplace = True)


In [ ]:
df_merge1.head()

,bag_of_words
title,
Toy Story (1995),adventure animation children comedy fantasy to...
Jumanji (1995),adventure children fantasy robinwilliams jonat...
Grumpier Old Men (1995),comedy romance waltermatthau jacklemmon sophia...
Waiting to Exhale (1995),comedy drama romance whitneyhouston angelabass...
Father of the Bride Part II (1995),comedy stevemartin dianekeaton martinshort alb...


In [ ]:
# instantiating and generating the count matrix
count_vect = CountVectorizer()
count_vect_matrix = count_vect.fit_transform(df_merge1['bag_of_words'])

In [ ]:
indices = pd.Series(df_merge1.index)

In [ ]:
# generating the cosine similarity matrix
sim_score = cosine_similarity(count_vect_matrix, count_vect_matrix)

In [ ]:
sim_score

array([[1.        , 0.20044593, 0.15811388, ..., 0.06681531, 0.        ,
        0.15075567],
       [0.20044593, 1.        , 0.        , ..., 0.14285714, 0.        ,
        0.16116459],
       [0.15811388, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.06681531, 0.14285714, 0.        , ..., 1.        , 0.        ,
        0.0805823 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.15075567, 0.16116459, 0.        , ..., 0.0805823 , 0.        ,
        1.        ]])

In [ ]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, sim_score = sim_score):
    
    recommended_movies = []
    
    # getting the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(sim_score[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df_merge1.index)[i])
        
    return recommended_movies

In [ ]:
recommendations('Waiting to Exhale (1995)')

['Melbourne (2014)',
 'Prime (2005)',
 'One I Love, The (2014)',
 '12 Storeys (Shier lou) (1997)',
 'Thanks for Sharing (2012)',
 'Kahlekuningas (2002)',
 'Being Flynn (2012)',
 'Love Is All You Need (Den skaldede frisør) (2012)',
 'Opposite Day (2009)',
 'Woman, a Gun and a Noodle Shop, A (San qiang pai an jing qi) (2009)']

<a id="four"></a>
## Explotory Data Analysis

<a id="five"></a>
## Modeling

<a id="six"></a>
## Model Performance

<a id="seven"></a>
## Model Explanations